# ML-mетрики качества

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Hit rate

* **Hit rate** = был ли хотя бы 1 релевантный товар среди рекомендованных
* **Hit rate@k** = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

In [2]:
recommended_list = [1432, 156, 1134, 27,
                    1543, 3345, 533, 11, 43, 32]  # id товаров
bought_list = [521, 32, 143, 991]

In [3]:
def hit_rate(recommended_list, bought_list, top_k: int = None):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    if top_k:
        if top_k > len(recommended_list)| top_k is None:
            top_k = len(recommended_list)
    flags = np.isin(bought_list, recommended_list[:top_k])

    return (flags.sum() > 0) * 1

In [4]:
print(f'Hit_rate = {hit_rate(recommended_list, bought_list)}\n'
      f'Hit rate@5 = {hit_rate(recommended_list, bought_list, top_k=5)}')

Hit_rate = 1
Hit rate@5 = 0


## Precision
**Precision** - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров юзер купил
* Precision= (# of recommended items that are relevant) / (# of recommended items)  
* Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)
* Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  


In [5]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]
prices_recommended = [400, 60, 40, 40, 90, 13, 18, 24, 120, 140]

In [6]:
def precision(recommended_list, bought_list, top_k: int = None):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    if top_k:
        if top_k > len(recommended_list) | top_k is None:
            top_k = len(recommended_list)
    flags = np.isin(bought_list, recommended_list[:top_k])

    return flags.sum() / len(recommended_list[:top_k])

In [7]:
def money_precision(recommended_list, bought_list, prices_recommended, top_k: int = None):

    recommend_list = np.array(recommended_list)
    bought_list = np.array(bought_list)
    prices_recommended = np.array(prices_recommended)
    

    if top_k:
        if top_k > len(recommended_list)| top_k is None:
            top_k = len(recommended_list)
        
    flags = np.isin(recommend_list[:top_k], bought_list)
    precision = np.dot(flags, prices_recommended[:top_k]).sum() / \
        prices_recommended[:top_k].sum()

    return precision

In [8]:
print(f'precision = {precision(recommended_list, bought_list)}\n'
      f'precision@5 = {precision(recommended_list, bought_list, top_k=5)}\n'
      f'money_precision@5 = {money_precision(recommended_list, bought_list, prices_recommended, top_k=5)}')

precision = 0.2
precision@5 = 0.4
money_precision@5 = 0.6984126984126984


## Recall
**Recall** - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендова
* Recall= (# of recommended items that are relevant) / (# of relevant items)

* Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

* Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)

Note: в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [9]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
prices_recommended = [400, 60, 40, 40, 90, 13, 18, 24, 120, 140]
bought_list = [521, 32, 143, 991]
prices_bought = [301, 202, 400, 40]

In [10]:
def recall(recommended_list, bought_list, top_k: int = None):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    if top_k:
        if top_k > len(recommended_list) | top_k is None:
            top_k = len(recommended_list)
    flags = np.isin(bought_list, recommended_list[:top_k])

    return flags.sum() / len(bought_list)

In [11]:
def money_recall(recommended_list, bought_list, prices_recommended, prices_bought, top_k: int = None):

    bought_list = np.array(bought_list)
    prices_bought = np.array(prices_bought)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    if top_k:
        if top_k > len(recommended_list) | top_k is None:
            top_k = len(recommended_list)

    flags = np.isin(recommended_list[:top_k], bought_list)
    recall = np.dot(flags, prices_recommended[:top_k]).sum() / prices_bought.sum()

    return recall

In [12]:
print(f'precision = {recall(recommended_list, bought_list)}\n'
      f'precision@3 = {recall(recommended_list, bought_list, top_k=3)}\n'
      f'money recall@10) = {money_recall(recommended_list, bought_list, prices_recommended, prices_bought, top_k=10)}')

precision = 0.5
precision@3 = 0.25
money recall@10) = 0.46659597030752914


***********

# Метрики ранжирования
**Если важен порядок рекомендаций.**

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [13]:
recommended_list = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  # id товаров
bought_list = [1, 3, 5, 7, 9, 11]

In [14]:
def ap(recommended_list, bought_list, top_k: int = None):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    if top_k:
        if top_k > len(recommended_list) | top_k is None:
            top_k = len(recommended_list)

    relevant_indexes = np.nonzero(np.isin(recommended_list[:top_k], bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0

    amount_relevant = len(relevant_indexes)

    sum_ = sum([precision(recommended_list, bought_list, top_k=index_relevant+1)
                for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [15]:
print(f'AP@8 = {ap(recommended_list, bought_list, 8)}')

AP@8 = 0.6083333333333333


## MAP@k
​
MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций
​
$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [16]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1,2]] # юзер 3

In [17]:
def map_at_k(recommended_list, bought_list, top_k: int = None):
    
    sum_ap = 0
    n_users = len(recommended_list)
    if n_users == 0:
        return 0
    for user in range(n_users):
        sum_ap += ap(recommended_list[user], bought_list[user], top_k)

    return sum_ap / n_users

In [18]:
print(f'MAP@5 = {map_at_k(recommended_list_3_users, bought_list_3_users, 5)}')

MAP@5 = 0.3333333333333333


## AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

## Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

In [19]:
recommended_list = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  # id товаров
bought_list = [1, 3, 5, 7, 9, 11]

In [20]:
def ndcg_at_k(recommended_list, bought_list, top_k):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    if top_k > len(recommended_list):
        top_k = len(recommended_list)
    flags = np.isin(recommended_list[:top_k], bought_list)
    gain_array = [1/i if i < 3 else 1/np.log2(i) for i in range(1, top_k+1)]
    ideal_dcg = sum(gain_array) / top_k # top_k можно убрать, оставил на случай если надо будет выводить значение  DCG
    dcg = np.dot(flags, np.array(gain_array)) / top_k

    return dcg / ideal_dcg

In [21]:
print(f'NDCG@5 = {ndcg_at_k(recommended_list, bought_list, 5)}')

NDCG@5 = 0.510061109328546


## Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [22]:
def reciprocal_rank(recommended_list, bought_list, top_k):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    sum_ku = 0

    if top_k > len(recommended_list[0]):
        top_k = len(recommended_list[0])
    
    n_users = len(recommended_list)
    if n_users == 0:
        return 0
    
    for user in range(n_users):
        try:
            ku = np.nonzero(np.isin(recommended_list[user][:top_k], bought_list[user]))[0][0] + 1
            sum_ku += 1 / ku
        except IndexError:
            continue

    return sum_ku/n_users

In [23]:
print(f'MRR@5 = {reciprocal_rank(recommended_list_3_users, bought_list_3_users, 5)}')

MRR@5 = 0.3333333333333333


*********************
# 
**Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

## Метрики на основе ранговой корреляции

Отдельно стоит выделить метрики качества ранжирования, основанные на одном из коэффициентов ранговой корреляции. В статистике, ранговый коэффициент корреляции — это коэффициент корреляции, который учитывает не сами значения, а лишь их ранг (порядок). 


* Ранговый коэффициент корреляции Кендэлла

* Ранговый коэффициент корреляции Спирмена

In [24]:
from scipy.stats import spearmanr
from scipy.stats import kendalltau  

print(spearmanr(recommended_list[:len(bought_list)], bought_list), kendalltau(
    recommended_list[:len(bought_list)], bought_list), sep='\n')

SpearmanrResult(correlation=0.7714285714285715, pvalue=0.07239650145772594)
KendalltauResult(correlation=0.6, pvalue=0.1361111111111111)


## Метрики на основе каскадной модели поведения

**Учитывают зависимость простотра одного элемента от просмотров других элементов**

Подобные модели поведения пользователя, где изучение предложенных ему элементов происходит последовательно и вероятность просмотра элемента зависит от релевантности предыдущих называются каскадным.

В расчетах используются вероятности

* Expected reciprocal rank (ERR)
* PFound